# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [34]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [44]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [36]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [37]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [38]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [39]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [97]:
def get_links(url):
    try:
        website = Website(url)
        openai = OpenAI(
            base_url='http://localhost:11434/v1',
            api_key='ollama',
            timeout=30  # Add timeout
        )
        response = openai.chat.completions.create(
            model="llama3.2",  # Fixed model name
            messages=[
                {"role": "system", "content": link_system_prompt},
                {"role": "user", "content": get_links_user_prompt(website)}
            ],
        )
        result = response.choices[0].message.content
        try:
            return json.loads(result)
        except json.JSONDecodeError:
            print(f"Invalid JSON response: {result}")
            return {"links": []}
            
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")
        return {"links": []}

In [98]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/tencent/Hunyuan-A13B-Instruct',
 '/google/gemma-3n-E4B-it',
 '/OmniGen2/OmniGen2',
 '/google/magenta-realtime',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/OmniGen2/OmniGen2',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/multimodalart/self-forcing',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/facebook/seamless-interaction',
 '/datasets/FreedomIntelligence/ShareGPT-4o-Image',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/

In [99]:
get_links("https://huggingface.co")

Invalid JSON response: {
    "links": [
        {"type": "About page", "url": "https://huggingface.co/"},
        {"type": "Blog", "url": "https://blog.huggingface.co/"},
        {"type": "GitHub repository", "url": "https://github.com/huggingface"},
        {"type": "Twitter handle", "url": "https://twitter.com/huggingface"},
        {"type": "LinkedIn company page", "url": "https://www.linkedin.com/company/huggingface/"},
        {
            "type": "Documentation index",
            "url": "https://huggingface.co/docs"
        },
        {
            "type": "Changelog",
            "url": "https://changelog.huggingface.co/'
        },
        {"type": "Status page", "url": "https://status.huggingface.co/"},
        {"type": "Join Discord community", "url": "https://join.discord.com/huggingface"}
    ]
}


{'links': []}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [100]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [101]:
print(get_all_details("https://huggingface.co"))

Invalid JSON response: Here is the list of relevant links for a brochure about Hugging Face in JSON format:

{
    "links": [
        {"type": "home page", "url": "https://huggingface.co"},
        {"type": "company page", "url": "https://www.linkedin.com/company/huggingface/"},
        {"type": "blog page", "url": "https://blog.huggingface.co"},
        {"type": "discuss forum", "url": "https://discuss.huggingface.co"},
        {"type": "status page", "url": "https://status.huggingface.co/"},
        {"type": "github page", "url": "https://github.com/huggingface"},
        {"type": "twitter handle", "url": "https://twitter.com/huggingface"}
    ]
}
Found links: {'links': []}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and appl

In [127]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [118]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [119]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Invalid JSON response: Here are the relevant links for a brochure about Hugging Face in JSON format:
```
{
  "links": [
    {"type": "Overview/Company page", "url": "https://huggingface.co"},
    {"type": "About us/FAQ page", "url": "https://discuss.huggingface.co/"},
    {"type": "Blog", "url": "https://blog.huggingface.co/"},
    {"type": "Changelog", "url": "https://changelog.huggingface.co/"}
  ]
}
```
I excluded links related to model download (`/models`), dataset access (`/datasets`), logins/joins (`/login`, `/join`), pricing (`/pricing`), enterprise features, product announcements (e.g., `black-forest-labs`, `google/gemma-3n-E4B-it`), and links that are not publicly accessible or relevant to the company overview.
Found links: {'links': []}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n3 days ago\n•\n110k\n•\n1.07k\ntencent/Hunyuan-A13B-Instruct\nUpdated\nabout 4 hours ago\n•\n5.02k\n•\n635\ngoogle/gemma-3n-E4B-it\nUpdated\n4 days ago\n•\n90k\n•\n319\nOmniGen2/OmniGen2\nUpdated\n7 days ago\n•\n30.8k\n•\n281\ngoogle/magenta-realtime\nUpdated\n8 days ago\n•\n420\nBrowse 1M+ models\nSpaces\nRunning\n8.95k\n8.95k\nDeepSite v2\n🐳

In [120]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="llama3.2",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [121]:
create_brochure("HuggingFace", "https://huggingface.co")

Invalid JSON response: Here is the list of relevant links for a brochure about the company, Hugging Face:

{
  "links": [
    {
      "type": "About/Company page",
      "url": "https://huggingface.co"
    },
    {
      "type": "All team affiliations (e.g. allenai, facebook, google, etc.)",
      "url": [
        "https://allenai.org/",
        "https://facebook.com/",
        "https://google.com/",
        "https://intel.com/",
        "https://microsoft.com/"
      ]
    },
    {
      "type": "Documentation website",
      "url": "https://huggingface.co/docs"
    },
    {
      "type": "Changelog",
      "url": "changelog.huggingface.co"
    },
    {
      "type": "Blog",
      "url": "blog.huggingface.co"
    },
    {
      "type": "Discord server",
      "url": "https://discord.com/invite/huggingface"
    },
    {
      "type": "GitHub repository",
      "url": "https://github.com/huggingface"
    },
    {
      "type": "Twitter profile",
      "url": "https://twitter.com/hugging

# Hugging Face Brochure

## About Us

Welcome to Hugging Face, the AI community building the future. We are a collaborative platform where machine learning practitioners come together to share knowledge, develop models, and push the boundaries of what is possible with artificial intelligence.

## Mission

Our mission is to democratize access to cutting-edge machine learning technology, empowering users to create, discover, and collaborate on ML projects like never before. We achieve this by developing innovative tools, hosting a vast repository of high-quality datasets, and fostering a vibrant community of developers and researchers.

## Our Vision

Hugging Face envisions a world where AI is accessible, usable, and integrated into every aspect of life. We strive to make our technology the foundation upon which machine learning innovation is built, enabling individuals and organizations to unlock new possibilities and drive meaningful impact.

## Community

Our community is at the heart of Hugging Face's mission. With over 50,000 organizations using our platform, we have created a space where machine learning enthusiasts can share knowledge, resources, and expertise. Join us today and become part of a vibrant ecosystem that is shaping the future of AI.

## Our Tools and Services

We offer a wide range of tools and services designed to support your ML journey:

*   **Hugging Face Hub**: A comprehensive platform for developing, deploying, and sharing machine learning models.
*   **Datasets**: A vast repository of high-quality datasets for any ML task.
*   **Spaces**: A collaboration tool for hosting, sharing, and managing public models, datasets, and applications.

## Technology

We develop cutting-edge technology that powers the Hugging Face platform. Our flagship technologies include:

*   **Transformers**: State-of-the-art ML frameworks for PyTorch, TensorFlow, and JAX.
*   **Diffusers**: Cutting-edge Diffusion models in PyTorch.
*   **Safetensors**: A safe way to store and distribute neural network weights.

## Careers

Join our team and be part of revolutionizing the field of machine learning!

*   **Developer**: Contribute to our innovative technology and help shape the future of AI.
*   **Data Scientist**: Work with our vast repository of high-quality datasets and develop new applications.
*   **Engineering Manager**: Lead teams of engineers and drive innovation on our platform.

## Get Involved

Ready to join the Hugging Face community? Sign up today and start exploring our platform, sharing knowledge with others, and contributing to the next generation of AI.

[Learn More About Careers at Hugging Face](#)

[Discover Hugging Face's Blog and Resource Center](https://huggingface.co/blog)

Join us on social media and stay up-to-date on the latest developments in the world of machine learning:

*   **Twitter**: [Twitter Profile](https://twitter.com/)
*   **LinkedIn**: [LinkedIn Profile](https://www.linkedin.com/)
*   **GitHub**: [GitHub Profile](https://github.com/)

Hugging Face is constantly innovating and pushing the boundaries of what is possible with AI. Join us on this journey and be part of shaping the future of machine learning!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [122]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="llama3.2",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [123]:
stream_brochure("HuggingFace", "https://huggingface.co")

Invalid JSON response: Here are the relevant links for a brochure about the company, Hugging Face:

{
  "links": [
    {
      "type": "Company/Splash page",
      "url": "https://huggingface.co"
    },
    {
      "type": "About page",
      "url": "https://discuss.huggingface.co"
    },
    {
      "type": "Blog/News",
      "url": "https://blog.huggingface.co"
    },
    {
      "type": "GitHub Profile",
      "url": "https://github.com/huggingface"
    },
    {
      "type": "Twitter Handle",
      "url": "https://twitter.com/huggingface"
    },
    {
      "type": "LinkedIn Page",
      "url": "https://www.linkedin.com/company/huggingface/"
    }
  ]
}

I excluded the following links as they are not important for a brochure or are related to terms service and privacy, such as:

* `/terms-of-service`
* `/privacy`
* `mailto:press@huggingface.co`
* `/login`
Found links: {'links': []}


# Hugging Face
The AI Community Building the Future

## About Us
Hugging Face is a leading AI community building the future of machine learning. Our platform provides collaboration tools for developers to create, discover, and share models, datasets, and applications.

## What We Do
We provide a comprehensive suite of tools and services for machine learning development, including:

* **Models**: Host, collaborate on, and apply public models
* **Datasets**: Access and share datasets for any ML tasks
* **Spaces**: Collaborate on unlimited public models, datasets, and applications
* **Compute**: Deploy on optimized inference endpoints or update your Spaces applications to a GPU in a few clicks

## Community
Our community is at the heart of what we do. With over 50,000 organizations using our platform, we empower developers to build, share, and collaborate on AI models.

### Top Supporters

* **AI2**: Enterprise
* **Amazon**: Company
* **Google**: Company
* **Intel**: Company
* **Microsoft**: Company
* **Grammarly**: Team (company)

## Our Open Source Efforts
We are committed to building the foundation of ML tooling with our community. Our open source initiatives include:

* **Transformers**: State-of-the-art ML for PyTorch, TensorFlow, JAX
* **Diffusers**: State-of-the-art Diffusion models in PyTorch
* **Safetensors**: Safe way to store/distribute neural network weights
* **Hub Python Library**: Python client to interact with the Hugging Face Hub
* **Tokenizers**: Fast tokenizers optimized for research & production

## Join Our Journey
Join our community and join the future of machine learning. Explore our platform, collaborate on models and datasets, and get involved in our open source efforts.

### Careers
Join our team and contribute to shaping the future of AI.

### Contact Us
Get in touch with us at [insert contact information].

### Follow Us
Stay up-to-date with the latest news and developments from Hugging Face:

* Twitter: [insert Twitter handle]
* LinkedIn: [insert LinkedIn profile]
* GitHub: [insert GitHub profile]

In [126]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Invalid JSON response: Here is the list of relevant links in JSON format:

```
{
  "links": [
    {"type": "Company Overview", "url": "https://huggingface.co"},
    {"type": "GitHub Repository", "url": "https://github.com/huggingface"},
    {"type": "Twitter Handle", "url": "https://twitter.com/huggingface"},
    {"type": "LinkedIn Page", "url": "https://www.linkedin.com/company/huggingface/"},
    {"type": "Discord Channel", "url": "https://join.discord.huggingface.co"},
    {"type": "Zhi hu Organization Profile", "url": "https://www.zhihu.com/org/huggingface"},
    {"type": "Blog", "url": "https://discuss.huggingface.co"},
    {"type": "Status Page", "url": "https://status.huggingface.co/"},
    {"type": "Documentation Hub", "url": "https://docs.huggingface.co/"},
    {"type": "Learn About Us", "url": "https://huggingface.co/learn"}
  ]
}
```

I excluded the following types of links as they are not suitable for a brochure:

* Pricing pages
* Login pages
* Email addresses
* Terms of S

# Welcome to Hugging Face: Where ML Meets Love

[Image: A heartwarming illustration of a machine learning community coming together]

At Hugging Face, we believe that Artificial Intelligence (AI) has the power to drive meaningful connections and transformations in our lives. Our platform is designed to bring people and AI models together, fostering collaboration, innovation, and progress.

## Our Story

Hugging Face was founded by Francis Reynolds, Hubert Gardeleagues, and Hugo Lemoine, who shared a vision of creating an open-source platform for machine learning (ML) that would democratize access to cutting-edge technology. Today, we're proud to be the home of over 1 million publicly available models, datasets, and applications, with more than 50,000 organizations and companies using our platform.

## Our Community

We have a vibrant community of ML enthusiasts, researchers, engineers, and creators who contribute to our ecosystem through open-source projects, research papers, and collaborative initiatives. Our community is active on social media platforms such as Twitter, LinkedIn, GitHub, and Discord, making it easy for you to connect with like-minded individuals and stay up-to-date on the latest advancements in ML.

## Our Offerings

Hugging Face offers a comprehensive suite of tools and services designed to support your AI journey:

*   **Models**: Explore 1 million+ models from top universities, companies, and researchers.
*   **Datasets**: Browse 250k+ datasets for any ML task or project.
*   **Spaces**: Collaborate on unlimited public models, datasets, and applications.
*   **Compute**: Deploy optimized inference endpoints or update your Spaces applications to a GPU in seconds.

## Careers

Join our team of passionate individuals working towards revolutionizing AI. With career opportunities ranging from software engineering to research and product management, you'll have the chance to make a lasting impact on the future of ML.

[Image: A screenshot from our job posting page]

Ready to join the Hugging Face squad?

    [Contact Information](#)
    [Careers](https://www.huggingface.co/careers)
    #HuggingFace

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>